In [1]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
import numpy as np
import requests
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup 
from keras import layers
from keras import losses

In [2]:
import pandas as pd

train_df = pd.read_csv('/Users/jacobjohnson/data_sets/congressional_tweet_training_data.csv', names=['favorite_count', 'full_text', 'hashtags', 'retweet_count', 'year', 'party_id'], 
                    # dtype={'favorite_count': int, 'full_text': str, 'hashtags': str, 'retweet_count': int, 'year': int, 'party_id': str}, 
                    skipinitialspace=True, skiprows=1, sep=',')

test_df = pd.read_csv('/Users/jacobjohnson/data_sets/congressional_tweet_test_data.csv', names=['id', 'favorite_count_test', 'full_text_test', 'hashtags_test', 'retweet_count_test', 'year', 'party'], 
                    # dtype={'id': int, 'favorite_count': int, 'full_text': str, 'hashtags': str, 'retweet_count': int, 'year': int, 'party': str}, 
                    skipinitialspace=True, skiprows=1, sep=',')

train_df.pop('year')
test_df.pop('year')

# test_df.head()
train_df.head()

# target variable
# party = train_df.pop('party_id')



,favorite_count,full_text,hashtags,retweet_count,party_id
0,0,"b""RT @KUSINews: One of our longtime viewers wa...",KUSI,10,R
1,258,"b""Today I'm urging the @CDCgov to immediately ...",Coronavirus,111,R
2,0,"b'Tomorrow, #MO03 seniors graduate from Calvar...",MO03,2,R
3,9,b'Congrats to #TeamUSA and Canton Native @JGre...,TeamUSA WorldJuniors,3,R
4,3,b'Pleased to support @amergateways at their Ju...,ImmigrantHeritageMonth,3,D


In [3]:
train, val, test = np.split(train_df.sample(frac=1), [int(0.8*len(train_df)), int(0.9*len(train_df))])

print(len(train), 'training examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

474242 training examples
59280 validation examples
59281 test examples


In [4]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  df = dataframe.copy()
  names = df.pop('party_id')
  df = {key: value[:,tf.newaxis] for key, value in dataframe.items()}
  ds = tf.data.Dataset.from_tensor_slices((dict(df), names))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  ds = ds.prefetch(batch_size)
  return ds

In [10]:
# batch_size = 5
# train_ds = df_to_dataset(train, batch_size=batch_size)

batch_size = 256
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

/var/folders/nf/brcqg8zx7xg31hjn_xbd2jc80000gp/T/ipykernel_32165/2315459290.py:4: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key: value[:,tf.newaxis] for key, value in dataframe.items()}
/var/folders/nf/brcqg8zx7xg31hjn_xbd2jc80000gp/T/ipykernel_32165/2315459290.py:4: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key: value[:,tf.newaxis] for key, value in dataframe.items()}
/var/folders/nf/brcqg8zx7xg31hjn_xbd2jc80000gp/T/ipykernel_32165/2315459290.py:4: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key: value[:,tf.newaxis] for key, value in dataframe.items()

In [11]:
[(train_features, label_batch)] = train_ds.take(1)
print('Every feature:', list(train_features.keys()))
print('A batch of hashtags:', train_features['hashtags'])
print('A batch of targets:', label_batch )

Every feature: ['favorite_count', 'full_text', 'hashtags', 'retweet_count', 'party_id']
A batch of hashtags: tf.Tensor(
[[b'va01']
 [b'TradeSummit2018']
 [b'TN01 iloveTNteachers NationalTeacherAppreciationDay']
 [b'ClimateChange']
 [b'BoomerSooner']
 [b'Bronx']
 [b'MadeInAmerica']
 [b'NorthDakota Bakken']
 [b'LA04 shopsmall SmallBusinessWeek']
 [b'MN03']
 [b'NetNeutrality OneMoreVote SaveTheInternet']
 [b'TaxBill']
 [b'SmallBusiness']
 [b'opioids Missouri']
 [b'PatFinucane UnifinishedBusiness']
 [b'Willowbrook CA44']
 [b'Haiyan']
 [b'EidMubarak']
 [b'SOTU Ag SpaceForce AL03']
 [b'CCE']
 [b'Texas America FridayFeelting tcot']
 [b'MobileOffice']
 [b'POTUS']
 [b'cosgrads2019']
 [b'ObamaCare ObamaCare KlineTTH']
 [b'PA06']
 [b'NY13 ForThePeople']
 [b'Obamacare']
 [b'ATF']
 [b'COVID19']
 [b'WA']
 [b'SWFL']
 [b'ProtectingOurInfantsAct']
 [b'FamiliesBelongTogether']
 [b'CIR']
 [b'BetterOffNow']
 [b'TrumpSwamp ForThePeople']
 [b'nationalprayerbreakfast']
 [b'AEW15chat']
 [b'FundForAmericasKids

In [7]:
# functions for data preprocessing
# these have not been tailored to my data

# may use this one retweet_count & favorite_count
def get_normalization_layer(name, dataset):
  normalizer = layers.Normalization(axis=None)
  feature_ds = dataset.map(lambda x, y: x[name])
  normalizer.adapt(feature_ds)
  return normalizer

# may use this on full_text & hashtags
def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
  if dtype == 'string':
    # DO THESE REQUIRE ALL COLUMNS HAVE DTYPES??
    index = layers.StringLookup(max_tokens=max_tokens)
  else:
    index = layers.IntegerLookup(max_tokens=max_tokens)
  feature_ds = dataset.map(lambda x, y: x[name])
  index.adapt(feature_ds)
  encoder = layers.CategoryEncoding(num_tokens=index.vocabulary_size())
  return lambda feature: encoder(index(feature))

In [12]:
test_retweet_count_col = train_features['retweet_count']
test_retweet_count_layer = get_category_encoding_layer(name='retweet_count', dataset=train_ds, dtype='int')
test_retweet_count_layer(test_retweet_count_col)

<tf.Tensor: shape=(256, 5018), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [13]:
test_favorite_count_col = train_features['favorite_count']
test_favorite_count_layer = get_category_encoding_layer(name='favorite_count', dataset=train_ds, dtype='int')
test_favorite_count_layer(test_favorite_count_col)

<tf.Tensor: shape=(256, 7918), dtype=float32, numpy=
array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [8]:
test_full_text_col = train_features['full_text']
test_full_text_layer = get_category_encoding_layer(name='full_text', dataset=train_ds, dtype='string')
test_full_text_layer(test_full_text_col)

<tf.Tensor: shape=(5, 470945), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [9]:
test_hashtags_col = train_features['hashtags']
test_hashtags_layer = get_category_encoding_layer(name='hashtags', dataset=train_ds, dtype='string')
test_hashtags_layer(test_hashtags_col)

<tf.Tensor: shape=(5, 158956), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>